# EMT VT Integration and Post Analysis
### Author: Joel Meoak
### This notebook is used to integrate VT results into the EMT modern malware results using the VirusTotal Rest API and python module 

In [1]:
import vt
import nest_asyncio
import pandas as pd
import time

In [2]:
# Ensure hashes are viewable
pd.set_option('display.max_colwidth', 100)

In [3]:
API_KEY = '<API KEY>' # personal VT API key
csv_name = 'UNC2589_results.csv'
csv_path = 'K:\\development\\VMshare\\' + csv_name
csv_path

'K:\\development\\VMshare\\UNC2589_results.csv'

In [4]:
# Maintian session for vt module
nest_asyncio.apply()

In [5]:
report = pd.read_csv(csv_path)

In [6]:
report['VT Score'] = '0/70'

In [7]:
report

,Attack/Group,Model,SHA256 Hash,File Name,Detected,FileType,VT Score
0,BleedingBear_UACert,GBC1.joblib,0f9f31bbc69c8174b492cf177c2fbaf627fcdb5ac4473ca5589aa2be75cee735,0f9f31bbc69c8174b492cf177c2fbaf627fcdb5ac4473ca5589aa2be75cee735,0,exe,0/70
1,BleedingBear_UACert,GBC1.joblib,9d7c3463d4a4f4390313c214c7a79042b4525ae639e151b5ec8a560b0dd5bd0a,9d7c3463d4a4f4390313c214c7a79042b4525ae639e151b5ec8a560b0dd5bd0a,0,exe,0/70
2,BleedingBear_UACert,GBC1.joblib,29decd1e88b297aa67fef6e14e39889cfd2454c581b9371a1003b63a28324d0f,29decd1e88b297aa67fef6e14e39889cfd2454c581b9371a1003b63a28324d0f,1,exe,0/70
3,BleedingBear_UACert,TR1.joblib,0f9f31bbc69c8174b492cf177c2fbaf627fcdb5ac4473ca5589aa2be75cee735,0f9f31bbc69c8174b492cf177c2fbaf627fcdb5ac4473ca5589aa2be75cee735,0,exe,0/70
4,BleedingBear_UACert,TR1.joblib,9d7c3463d4a4f4390313c214c7a79042b4525ae639e151b5ec8a560b0dd5bd0a,9d7c3463d4a4f4390313c214c7a79042b4525ae639e151b5ec8a560b0dd5bd0a,0,exe,0/70
...,...,...,...,...,...,...,...
1027,Whispergate_Picusecurity,lgbm.joblib,dcbbae5a1c61dbbbb7dcd6dc5dd1eb1169f5329958d38b58c3fd9384081c9b78,dcbbae5a1c61dbbbb7dcd6dc5dd1eb1169f5329958d38b58c3fd9384081c9b78,0,exe,0/70
1028,Whispergate_Picusecurity,lgbm.joblib,a196c6b8ffcb97ffb276d04f354696e2391311db3841ae16c8c9f56f36a38e92,a196c6b8ffcb97ffb276d04f354696e2391311db3841ae16c8c9f56f36a38e92,0,exe,0/70
1029,Whispergate_Picusecurity,lgbm.joblib,00bc665d96ecadc6beb2a9384773a70391f08f8e7a2876253f32ceec793eb728,00bc665d96ecadc6beb2a9384773a70391f08f8e7a2876253f32ceec793eb728,0,exe,0/70
1030,Whispergate_Picusecurity,lgbm.joblib,9ef7dbd3da51332a78eff19146d21c82957821e464e8133e9594a07d716d892d,9ef7dbd3da51332a78eff19146d21c82957821e464e8133e9594a07d716d892d,0,exe,0/70


In [8]:
name = report['File Name']
name = name.drop_duplicates()
name

0      0f9f31bbc69c8174b492cf177c2fbaf627fcdb5ac4473ca5589aa2be75cee735
1      9d7c3463d4a4f4390313c214c7a79042b4525ae639e151b5ec8a560b0dd5bd0a
2      29decd1e88b297aa67fef6e14e39889cfd2454c581b9371a1003b63a28324d0f
24     14736be09a7652d206cd6ab35375116ec4fad499bb1b47567e4fd56dcfcd22ea
32     d0aad99f10bdd6f6af2f7a0f6c319ed7d126de4d1ff44ca86858e7ffc17cc39b
                                     ...                               
968    ff3b45ecfbbdb780b48b4c829d2b6078d8f7673d823bedbd6321699770fa3f84
969    9cdaacaba35c3a473ec5b652d035a9593ee822609e79662223869e2b7298dc0a
970    35ab54a9502e975c996cbaee3d6a690da753b4af28808d3be2054f8a58e5c7c5
973    00bc665d96ecadc6beb2a9384773a70391f08f8e7a2876253f32ceec793eb728
975    bbe1949ffd9188f5ad316c6f07ef4ec18ba00e375c0e6c2a6d348a2a0ab1e423
Name: File Name, Length: 125, dtype: object

In [9]:
hashlist = report['SHA256 Hash']
hashlist_no_duplicates = hashlist.drop_duplicates()
len(hashlist_no_duplicates)

125

In [ ]:
with vt.Client(API_KEY) as client:
    for hash in hashlist_no_duplicates:
        try:
            # pulls the full report of any malware assciated with hash
            response = client.get_object(f"/files/{hash}")
            response_dict = response.to_dict()
            file_stats = response_dict['attributes']['last_analysis_stats']
            malicious_count =  file_stats['malicious']
            total_count =  file_stats['malicious'] + file_stats['undetected']
            report_stats = str(malicious_count) + '/' +  str(total_count)
            report.loc[report['SHA256 Hash'] == hash, 'VT Score'] = report_stats
            print(hash + ' ' + report_stats)
        except vt.APIError as e:
            print(f"Error with hash {hash}: {e}")
            report.loc[report['SHA256 Hash'] == hash, 'VT Score'] = 'Not Found'
        time.sleep(20) # Without premium API key, limited to 4 per min

In [29]:
report

,Attack/Group,Model,SHA256 Hash,File Name,Detected,FileType,VT Score
0,BleedingBear_UACert,GBC1.joblib,0f9f31bbc69c8174b492cf177c2fbaf627fcdb5ac4473ca5589aa2be75cee735,0f9f31bbc69c8174b492cf177c2fbaf627fcdb5ac4473ca5589aa2be75cee735,0,exe,0/72
1,BleedingBear_UACert,GBC1.joblib,9d7c3463d4a4f4390313c214c7a79042b4525ae639e151b5ec8a560b0dd5bd0a,9d7c3463d4a4f4390313c214c7a79042b4525ae639e151b5ec8a560b0dd5bd0a,0,exe,58/70
2,BleedingBear_UACert,GBC1.joblib,29decd1e88b297aa67fef6e14e39889cfd2454c581b9371a1003b63a28324d0f,29decd1e88b297aa67fef6e14e39889cfd2454c581b9371a1003b63a28324d0f,1,exe,52/72
3,BleedingBear_UACert,TR1.joblib,0f9f31bbc69c8174b492cf177c2fbaf627fcdb5ac4473ca5589aa2be75cee735,0f9f31bbc69c8174b492cf177c2fbaf627fcdb5ac4473ca5589aa2be75cee735,0,exe,0/72
4,BleedingBear_UACert,TR1.joblib,9d7c3463d4a4f4390313c214c7a79042b4525ae639e151b5ec8a560b0dd5bd0a,9d7c3463d4a4f4390313c214c7a79042b4525ae639e151b5ec8a560b0dd5bd0a,0,exe,58/70
...,...,...,...,...,...,...,...
1027,Whispergate_Picusecurity,lgbm.joblib,dcbbae5a1c61dbbbb7dcd6dc5dd1eb1169f5329958d38b58c3fd9384081c9b78,dcbbae5a1c61dbbbb7dcd6dc5dd1eb1169f5329958d38b58c3fd9384081c9b78,0,exe,62/72
1028,Whispergate_Picusecurity,lgbm.joblib,a196c6b8ffcb97ffb276d04f354696e2391311db3841ae16c8c9f56f36a38e92,a196c6b8ffcb97ffb276d04f354696e2391311db3841ae16c8c9f56f36a38e92,0,exe,59/70
1029,Whispergate_Picusecurity,lgbm.joblib,00bc665d96ecadc6beb2a9384773a70391f08f8e7a2876253f32ceec793eb728,00bc665d96ecadc6beb2a9384773a70391f08f8e7a2876253f32ceec793eb728,0,exe,51/70
1030,Whispergate_Picusecurity,lgbm.joblib,9ef7dbd3da51332a78eff19146d21c82957821e464e8133e9594a07d716d892d,9ef7dbd3da51332a78eff19146d21c82957821e464e8133e9594a07d716d892d,0,exe,55/69


In [30]:
newCsvName = 'final_' + csv_name
report.to_csv(newCsvName, index=True)